In [1]:
import pandas as pd
import re
from tqdm import tqdm
import torch
import torchmetrics
import ast
import random
import transformers
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
import tree
import functions
from functions import Dataset_create_embeddings_from_tokens_for_text, Clinical_Bert_Model, Dataset_create_embeddings_from_classdescripsion_for_classsentence_list
import sequence_label
import wandb
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import multiprocessing as mp
import argparse_para

/u/home/niur/venv_1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/home/niur/venv_1/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
args = argparse_para.arg_parser()

In [3]:
tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease = tree.build_a_tree()
# build the tree, it is four dicts.

100%|██████████| 1391/1391 [00:00<00:00, 57202.16it/s]


In [4]:

tokenizer = transformers.BertTokenizer.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b', ignore_mismatched_sizes=True)
clinical_DS_bert = transformers.BertModel.from_pretrained('./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b')
# get the pre-trained bert model.   

Some weights of the model checkpoint at ./Model_Bio_Clinical_DS_BERT/Bio_DischargeSummary_BERT/hub/models--emilyalsentzer--Bio_Discharge_Summary_BERT/snapshots/affde836a50e4d333f15dae9270f5a856d59540b were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print('GPU', torch.cuda.is_available())

clinical_Bert_Model = Clinical_Bert_Model(clinical_DS_bert=clinical_DS_bert)
clinical_Bert_Model = clinical_Bert_Model.to(device)

GPU True


In [6]:
# this dataframe contains the incd9 classes embeddings

df_classes_descripsion = functions.create_classes_descripsions(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
# df_classes_descripsion_embeddings has two columns: 'classes' and 'descripsion', and the length of it is all the category and subclass(end tokens not included).

class_input_ids_tensor = functions.create_class_bert_tokens(df_classes_descripsion, tokenizer)  # , class_token_type_idstensor, class_attention_mask_tensor
class_input_ids_tensor = class_input_ids_tensor.detach()
create_class_embeddings_dataset = Dataset_create_embeddings_from_classdescripsion_for_classsentence_list(class_input_ids_tensor)  # , class_token_type_idstensor, class_attention_mask_tensor
create_class_embeddings_dataloader = DataLoader(create_class_embeddings_dataset, 1, shuffle=False, drop_last=False, num_workers=8)
class_embeddings_tensor = functions.create_classes_embeddings(create_class_embeddings_dataloader, clinical_Bert_Model, device)
print('the shape of class_embedding_tensor is: ', class_embeddings_tensor.shape)
class_embeddings_dict = functions.create_class_embeddings_dict(list(df_classes_descripsion['classes']), class_embeddings_tensor)
print('the length of class embedding is(end tokens not included)',len(class_embeddings_dict))
# keys are classes and values are embeddings tensors, and the length of it is all the category and subclass(end tokens not included).
# the shape of embeddings is (n, 768)
torch.cuda.empty_cache()

100%|██████████| 15978/15978 [41:44<00:00,  6.38it/s] 


the shape of class_embedding_tensor is:  torch.Size([15978, 768])
the length of class embedding is(end tokens not included) 15978


In [7]:
print(class_embeddings_tensor.device)

cuda:0


In [8]:
df_train_top500 = pd.read_csv('./data/top500_datasets/trainset_top500_new.csv')
df_validation_top500 = pd.read_csv('./data/top500_datasets/validationset_top500_new.csv')

In [9]:
df_train_top500 = functions.text_cleaning(df_train_top500)
df_validation_top500 = functions.text_cleaning(df_validation_top500)
df_train_top500 = functions.convert_icdstr_to_list(df_train_top500)
df_validation_top500 = functions.convert_icdstr_to_list(df_validation_top500)
df_train_top500 = sequence_label.create_sequence_label_depth_first(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, df_train_top500)
df_validation_top500 = sequence_label.create_sequence_label_depth_first(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, df_validation_top500)
print(df_train_top500.columns)
print(df_train_top500.shape, df_validation_top500.shape)


/u/home/niur/htc_mimic3/functions.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub("(Admission Date:)|(Discharge Date:)|(Service:)|(Date of Birth:)|(Sex:)|(Attending:)|(Provider:)|(Name:)|(Date/Time:)|(MD Phone:)|(Completed by:)|(Job#:)|(Dictated By:)|(D: )|(T: )|(JOB :)", "", dataframe['TEXT'].iloc[i])
/u/home/niur/htc_mimic3/functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['TEXT'].iloc[i] = re.sub(r'\[\*\*.*?\*\*\]', '', dataframe['TEXT'].iloc[i])  # delete [**      **]
/u/home/niur/htc_mimic3/functions.py:13: SettingWithCopyWarning: 
A value is trying to

Index(['HADM_ID', 'TEXT', 'ICD9_CODE', 'SEQUENCIAL_LABEL'], dtype='object')
(8286, 4) (1468, 4)


In [10]:


input_ids_list_train = functions.create_text_tokens(df_train_top500, tokenizer) # , token_type_idstensor_train, attention_mask_tensor_train
# input_ids_list_train = input_ids_list_train.detach()
print('train token shape:', len(input_ids_list_train))
input_ids_list_validation = functions.create_text_tokens(df_validation_top500, tokenizer) # , token_type_idstensor_validation, attention_mask_tensor_validation
# input_ids_list_validation = input_ids_list_validation.detach()
print('valid token shape:', len(input_ids_list_validation))

# train_dataset_top500_create_text_embeddings = Dataset_create_embeddings_from_tokens_for_text(input_ids_tensor_train) # , token_type_idstensor_train, attention_mask_tensor_train
# validation_datasdet_top500_create_text_embeddings = Dataset_create_embeddings_from_tokens_for_text(input_ids_tensor_validation) #, token_type_idstensor_validation, attention_mask_tensor_validation
# train_dataloader_top500_create_text_embeddings = DataLoader(train_dataset_top500_create_text_embeddings, 1, shuffle=False, drop_last=False, num_workers=8)
# validation_dataloader_top500_create_text_embeddings = DataLoader(validation_datasdet_top500_create_text_embeddings, 1, shuffle=False, drop_last=False, num_workers=8)


# train_text_embeddings_tensor = functions.create_text_embeddings(train_dataloader_top500_create_text_embeddings, clinical_Bert_Model, device)
# validation_text_embeddings_tensor = functions.create_text_embeddings(validation_dataloader_top500_create_text_embeddings, clinical_Bert_Model, device)
# # this is the outputs of encoder and will be the cross attention input of decoder.
# print('train text embedding shape:', train_text_embeddings_tensor.shape)
# print('valid text embedding shape:', validation_text_embeddings_tensor.shape)

100%|██████████| 8286/8286 [04:25<00:00, 31.26it/s]


train token shape: 8286


100%|██████████| 1468/1468 [00:45<00:00, 32.04it/s]

valid token shape: 1468


In [11]:
mask_dict = functions.create_tokens_mask_dict(tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
# it s a dictionary contains the -inf and 0 mask(1d tensor) of each category and subclass.
class_label_index_dict = functions.create_class_label_index_dict(mask_dict)
# {'start':0, .......}
index_to_class_label_dict = functions.create_index_to_class_label_dict(mask_dict)
# {0:'start', .......}
print('the length of mask dict is:',len(class_label_index_dict))

the length of mask dict is: 17392


In [12]:
train_sequencial_label_list = list(df_train_top500['SEQUENCIAL_LABEL'])       # [[L], [L], [L]......]
valiation_sequencial_label_list = list(df_validation_top500['SEQUENCIAL_LABEL'])
# here the data pass into the dataset can not be tensors, because change into embeddings need the complete class embedding dict.

train_label_index_list = functions.convrt_sequencial_label_to_label_index(class_label_index_dict, train_sequencial_label_list)  # [tensor(L), tensor(L), tensor(L), tensor(L), tensor(L).......]
validation_label_index_list = functions.convrt_sequencial_label_to_label_index(class_label_index_dict, valiation_sequencial_label_list)

In [13]:
class Dataset_transformer(Dataset):
    def __init__(self, input_ids_list_train, label_index_list):
        self.input_ids_list_train = input_ids_list_train   #[(n,128), (n,128), (n,128), (n,128), (n,128), (n,128), (n,128)]
        self.label_index_list = label_index_list      # [tensor(L), tensor(L), tensor(L), tensor(L), tensor(L).......]

    def __len__(self):
        return len(self.input_ids_list_train)

    def __getitem__(self, index):
        return self.input_ids_list_train[index], self.label_index_list[index]
    
def collate_fn(batch):
    batch_input_ids_list = [item[0] for item in batch]  #[(n,128), (n,128), (n,128), (n,128), (n,128), (n,128), (n,128)]
    batch_input_ids_tensor = torch.nn.utils.rnn.pad_sequence(batch_input_ids_list, batch_first=True)  # (b,n,128)  n is the largest number of sentences in a samples batch.
    encoder_output_mask = []
    for item in batch_input_ids_list:
        single_mask = torch.tensor([True]*batch_input_ids_tensor.shape[1])  # masked is True
        single_mask[:item.shape[0]] = False    # not masked is False
        encoder_output_mask.append(single_mask)
    encoder_output_mask = torch.stack(encoder_output_mask) # (b,n)
    
    
    batch_label_index = torch.nn.utils.rnn.pad_sequence([item[1] for item in batch], batch_first=True, padding_value=17391)  # (b,L) L is the largest length in this batch
    max_L = batch_label_index.shape[1]
    padding_mask = []
    for item in batch:
        single_mask = torch.tensor([float('-inf')]*max_L)  # masked is True
        true_L = item[1].shape[0]
        single_mask[:true_L] = 0.0    # not masked is False
        padding_mask.append(single_mask)
    padding_mask = torch.stack(padding_mask)   # (b,L) L is the largest length in this batch   (0,-inf)

    
    return batch_input_ids_tensor, encoder_output_mask, batch_label_index, padding_mask     #  (b,n,128) (b,n) (b,L) (b,L)  L is the largest length in this batch 


train_dataset = Dataset_transformer(input_ids_list_train, train_label_index_list)
validation_dataset = Dataset_transformer(input_ids_list_validation, validation_label_index_list)
batch_size_transformer = args.batch_size_transformer
num_workers = args.num_workers
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size_transformer, shuffle=True, drop_last=True, num_workers=num_workers, collate_fn=collate_fn, pin_memory=True)  
validation_dataloader = DataLoader(dataset=validation_dataset, batch_size=batch_size_transformer, shuffle=False, drop_last=False, num_workers=num_workers, collate_fn=collate_fn, pin_memory=True)
decoder_num_layers = args.decoder_num_layers
transformer_dropout = args.transformer_dropout
transformer_nhead = args.transformer_nhead

In [14]:
class Transformer_Model(torch.nn.Module):
    def __init__(self):
        super(Transformer_Model, self).__init__()

        self.clinical_bert = clinical_DS_bert
        for lay in self.clinical_bert.encoder.layer[args.bert_freeze_layerindex_start:args.bert_freeze_layerindex_end+1]:
            for para in lay.parameters():
                para.requires_grad = False

        self.class_embeddings_start = torch.nn.Parameter(torch.randn(768))
        self.class_embeddings_end_start = torch.nn.Parameter(torch.randn(768))

        self.decoder_layer = torch.nn.TransformerDecoderLayer(d_model=768, nhead=transformer_nhead, dropout=transformer_dropout, activation='gelu', batch_first=True)
        self.transformer_decoder = torch.nn.TransformerDecoder(self.decoder_layer, num_layers=decoder_num_layers)  # forward padding mask(N,L)  attention causal mask(L,L)  multiheadattention  is_causal

        self.linear1 = torch.nn.Linear(768, 768)
        self.gelu = torch.nn.GELU()
        self.linear2 = torch.nn.Linear(768, 768)
        self.softmax = torch.nn.Softmax()

    def forward(self, class_embeddings_dict, mask_dict, batch_input_ids_tensor, batch_label_index, attention_mask, batch_padding_mask, encoder_output_mask):   #  (b,n,128), (b,L)  (b,L)  (b,n)

        class_embeddings_dict = functions.add_end_classes_embeddings(class_embeddings_dict, self.class_embeddings_start, self.class_embeddings_end_start, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease, device)
        class_embedding_list = []
        for item in list(mask_dict.keys())[:-1]:
            class_embedding_list.append(class_embeddings_dict[item])
        class_embeddings_tensor = torch.stack(class_embedding_list)    # (C,768) C:17391

        label_embedding_tensor = functions.convert_sequencial_label_to_embedding_tensor(batch_label_index, class_embeddings_dict, index_to_class_label_dict)  # (b,L) --> (b,L,768)
        
        batch_text_embedding_tensor = torch.stack([self.clinical_bert(single_input_ids_tensor)['last_hidden_state'][:,0,:] for single_input_ids_tensor in batch_input_ids_tensor])   # (b,n,768)
        decoder_output = self.transformer_decoder(label_embedding_tensor, batch_text_embedding_tensor, tgt_mask=attention_mask, tgt_key_padding_mask=batch_padding_mask, memory_key_padding_mask=encoder_output_mask)#  (b,L,768)  (b,n,768) -->  (b,L,768)

        transformer_output = self.linear1(decoder_output)   #  (b,L,768)
        transformer_output = self.gelu(transformer_output)
        transformer_output = self.linear2(transformer_output)  #  (b,L,768)

        transformer_output = torch.nn.functional.normalize(transformer_output, dim=-1)  # (b,L,768)
        class_embeddings_tensor = torch.nn.functional.normalize(class_embeddings_tensor, dim=-1)  #  (C,768)

        # score_tensor = torch.nn.functional.cosine_similarity(transformer_output.unsqueeze(0).permute((1,0,2)), class_embeddings_tensor.unsqueeze(0), dim=-1)  # (L,1,768) (1,C,768) --> (L,C)
        score_tensor = torch.matmul(transformer_output, class_embeddings_tensor.T).to(dtype=torch.float16)  # (b,L,768), (768,C)  -->  (b,L,C)
        batch_mask_list = []
        for item in batch_label_index:  # (b,L)
            batch_mask_list.append(torch.stack([mask_dict[index_to_class_label_dict[index.item()]] for index in item]))   # (L,C)
        batch_mask_tensor = torch.stack(batch_mask_list)   # (b,L,C)
        batch_mask_tensor = batch_mask_tensor.to(device)
        logits = (score_tensor+batch_mask_tensor)   # (b,L,C)
        return logits  # (b,L,C)
        

In [15]:
transformer_model = Transformer_Model()
transformer_model = transformer_model.to(device)
learning_rate = args.learning_rate
criterion = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=17391)    # when compute, the size must be (N, C, L) and (N, L)
optimizer = torch.optim.AdamW(transformer_model.parameters(), lr=learning_rate, weight_decay=args.adam_weight_decay)

In [18]:
def valid(validation_dataloader):
    transformer_model.eval()
    print('Validation start...')
    with torch.no_grad():
        validation_loss = 0
        category_accuracy = 0
        first_class_accuracy = 0
        second_class_accuracy = 0
        disease_accuracy = 0
        label_f1_0_1_tensor = torch.tensor([])
        predicted_f1_0_1_tensor = torch.tensor([])
        label_AUROC_tesnor = torch.tensor([],dtype=int).to(device)
        predicted_AUROC_tensor = torch.tensor([]).to(device)
        random_outout_index = random.randint(0,1000)
        for i, (batch_input_ids_tensor, batch_encoder_output_mask, batch_label_index, batch_padding_mask) in tqdm(enumerate(validation_dataloader)):

            # torch.cuda.empty_cache()
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                batch_input_ids_tensor = batch_input_ids_tensor.to(device, non_blocking=True)
                batch_label_index = batch_label_index.to(device, non_blocking=True)
                batch_padding_mask = batch_padding_mask.to(device, non_blocking=True)
                batch_encoder_output_mask = batch_encoder_output_mask.to(device, non_blocking=True)
                attention_mask_valid = torch.triu(torch.full((batch_padding_mask.shape[1], batch_padding_mask.shape[1]), True), diagonal=1).to(device, non_blocking=True)   # masked is True, not masked is False
                

                batch_logits = transformer_model(class_embeddings_dict, mask_dict, batch_input_ids_tensor=batch_input_ids_tensor, batch_label_index=batch_label_index, attention_mask=attention_mask_valid, batch_padding_mask=batch_padding_mask, encoder_output_mask=batch_encoder_output_mask)   # (b,n,128), (b,L) --> (b,L,C)
                batch_logits = batch_logits.permute((0,2,1))   # --> (b,C,L)
                end_start_index_label = class_label_index_dict['/start'] 
                lable_for_loss_tensor = functions.create_lable_for_loss_tensor(batch_label_index, end_start_index_label, device) # (b,L)
                validation_loss += criterion(batch_logits, lable_for_loss_tensor)

                for j in range(batch_input_ids_tensor.shape[0]):
                    
                    single_input_ids_tensor_valid = batch_input_ids_tensor[j].unsqueeze(0)   # (1,n,128)
                    single_encoder_output_mask = batch_encoder_output_mask[j].unsqueeze(0)  # (1,n)

                    single_label_index_list = batch_label_index[j].tolist()    # [L]
                    padding_index = single_label_index_list.index(17391) if 17391 in single_label_index_list else len(single_label_index_list)
                    single_label_index_list = single_label_index_list[:padding_index]
                    single_sequencial_label = [index_to_class_label_dict[index] for index in single_label_index_list]

                    # ingle_label_index_tensor = label_index_list[j].unsqueeze(0)    # (1,L)
                    # single_text_embedding_tensor = single_text_embedding_tensor.to(device)
                    # single_label_index_tensor = single_label_index_tensor.to(device)

                    predicted_index_list = []
                    predicted_token_list = []
                    predicted_token_probabiluty_list = []
                    input_token_index_list = [class_label_index_dict['start']]
                    end_start_index_label = class_label_index_dict['/start'] 
                    while input_token_index_list[-1] != end_start_index_label:
                        logits = transformer_model(class_embeddings_dict, mask_dict, batch_input_ids_tensor=single_input_ids_tensor_valid, batch_label_index=torch.tensor([input_token_index_list]).to(device, non_blocking=True), attention_mask=None, batch_padding_mask=None, encoder_output_mask=single_encoder_output_mask)   # (1,n,128), (1,L) --> (1,L,C)
                        predicted_index = torch.argmax(logits.squeeze(0), dim=-1)[-1].item()   # (1,L,C)-->int
                        predicted_token_probabiluty_list.append(logits.squeeze(0)[-1])  # (1,L,C) -->[L,(C)]
                        predicted_index_list.append(predicted_index)   # [L]
                        predicted_token_list.append(index_to_class_label_dict[predicted_index])  # [L]
                        input_token_index_list.append(predicted_index)   # [L]

                        if len(predicted_token_list) > 150:
                            break

                    predicted_token_probabiluty_tensor = torch.nn.Softmax(dim=-1)(torch.stack(predicted_token_probabiluty_list))  # (L,C)
                    predicted_category_tokens_list, predicted_first_class_tokens_list, predicted_second_class_tokens_list, predicted_disease_tokens_list = functions.split_predicted_class_tokens(predicted_token_list, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
                    label_category_tokens_list, label_first_class_tokens_list, label_second_class_tokens_list, label_disease_tokens_list = functions.split_predicted_class_tokens(single_sequencial_label, tree_dict_start_to_category, tree_dict_category_to_first_subclass, tree_dict_first_subclass_to_second_subclass, tree_dict_second_subclass_to_disease)
                    category_accuracy += (len(set(predicted_category_tokens_list).intersection(set(label_category_tokens_list))) / max(len(predicted_category_tokens_list), len(label_category_tokens_list)))
                    first_class_accuracy += (len(set(predicted_first_class_tokens_list).intersection(set(label_first_class_tokens_list))) / max(len(predicted_first_class_tokens_list), len(label_first_class_tokens_list)))
                    second_class_accuracy += (len(set(predicted_second_class_tokens_list).intersection(set(label_second_class_tokens_list))) / max(len(predicted_second_class_tokens_list), len(label_second_class_tokens_list)))
                    disease_accuracy += (len(set(predicted_disease_tokens_list).intersection(set(label_disease_tokens_list))) / max(len(predicted_disease_tokens_list), len(label_disease_tokens_list)))

                    single_label_f1_0_1_tensor, single_predicted_f1_0_1_tensor = functions.create_0_1_tensor_for_f1(single_sequencial_label, predicted_disease_tokens_list)
                    label_f1_0_1_tensor = torch.cat([label_f1_0_1_tensor, single_label_f1_0_1_tensor])
                    predicted_f1_0_1_tensor = torch.cat([predicted_f1_0_1_tensor, single_predicted_f1_0_1_tensor])

                    single_probability_AUROC_predicted, single_label_AUROC_0_1_tensor = functions.create_probability_tensor_for_AUROC(single_sequencial_label, class_label_index_dict, predicted_token_probabiluty_tensor, predicted_index_list, device)
                    label_AUROC_tesnor = torch.cat([label_AUROC_tesnor, single_label_AUROC_0_1_tensor.to(device).type(torch.int)])
                    predicted_AUROC_tensor = torch.cat([predicted_AUROC_tensor, single_probability_AUROC_predicted])

                    if batch_size_transformer*i+j == random_outout_index:
                        print('label: ', single_sequencial_label)
                        print('predicted: ', predicted_token_list)



        label_f1_0_1_tensor = label_f1_0_1_tensor.reshape((df_validation_top500.shape[0], 14567))  # (N, C)
        predicted_f1_0_1_tensor = predicted_f1_0_1_tensor.reshape((df_validation_top500.shape[0], 14567))  # (N, C)
        valid_f1_micro = torchmetrics.functional.classification.multilabel_f1_score(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, threshold=0.5, average='micro', multidim_average='global')
        valid_f1_weighted = torchmetrics.functional.classification.multilabel_f1_score(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, threshold=0.5, average='weighted', multidim_average='global')


        label_AUROC_tesnor = label_AUROC_tesnor.reshape((df_validation_top500.shape[0], 17391))  # (N, C)
        predicted_AUROC_tensor = predicted_AUROC_tensor.reshape((df_validation_top500.shape[0], 17391))  # (N, C)
        valid_AUROC_micro = torchmetrics.functional.classification.multilabel_auroc(predicted_AUROC_tensor, label_AUROC_tesnor, num_labels=17391, average='micro')
        valid_AUROC_weighted = torchmetrics.functional.classification.multilabel_auroc(predicted_AUROC_tensor, label_AUROC_tesnor, num_labels=17391, average='weighted')

        valid_precision_micro = torchmetrics.functional.classification.multilabel_precision(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, multidim_average='global', average='micro')
        valid_recall_micro = torchmetrics.functional.classification.multilabel_recall(predicted_f1_0_1_tensor, label_f1_0_1_tensor, num_labels=14567, multidim_average='global', average='micro')

    return validation_loss/df_validation_top500.shape[0], category_accuracy/df_validation_top500.shape[0], first_class_accuracy/df_validation_top500.shape[0], second_class_accuracy/df_validation_top500.shape[0], disease_accuracy/df_validation_top500.shape[0], valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision_micro, valid_recall_micro





def train(train_dataloader, validation_dataloader, epochs=5):

    # config = { 'batch_size': batch_size_transformer, 'lr':learning_rate, 'loss':'CrossEntropyLoss', 'optim':'Adam', 
    #           'transformer_decoder_nhead':transformer_nhead, 'transformer_decoder_layers':decoder_num_layers, 'linear_layer':'768-768-Gelu-768', 'epochs':epochs}
    # wandb.init(project="Result", entity="htc-mimic3", config=config)

    print('Training start...')
    scaler = torch.cuda.amp.GradScaler()
    iter_gradient_accumulation = args.iter_gradient_accumulation

    for epoch in range(epochs):
        torch.cuda.empty_cache()
        print(f'Epoch: {epoch+1}')
        loss_training_epoch = 0

        transformer_model.train()
        for i, (batch_input_ids_tensor, batch_encoder_output_mask, batch_label_index, batch_padding_mask) in tqdm(enumerate(train_dataloader)):
            # torch.cuda.empty_cache()
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                loss_training_batch = 0
                batch_label_index = batch_label_index.to(device, non_blocking=True)  # (b,L)  L is the largest length in this batch 
                batch_padding_mask = batch_padding_mask.to(device, non_blocking=True)  # (b,L)  L is the largest length in this batch 
                batch_input_ids_tensor = batch_input_ids_tensor.to(device, non_blocking=True)  # (b,n,128) n is the largest number of sentenses in a sample
                batch_encoder_output_mask = batch_encoder_output_mask.to(device, non_blocking=True)  #  (b,n)  n is the largest number of sentenses in a sample
                

                attention_mask_train = torch.triu(torch.full((batch_padding_mask.shape[1], batch_padding_mask.shape[1]), True), diagonal=1).to(device, non_blocking=True)
                # masked is True, not masked is False
                batch_logits = transformer_model(class_embeddings_dict, mask_dict, batch_input_ids_tensor=batch_input_ids_tensor, batch_label_index=batch_label_index, attention_mask=attention_mask_train, batch_padding_mask=batch_padding_mask, encoder_output_mask=batch_encoder_output_mask)   # (b,4,768), (b,L) --> (b,L,C)
                batch_logits = batch_logits.permute((0,2,1))    # --> (b,C,L)
                end_start_index_label = class_label_index_dict['/start'] 
                lable_for_loss_tensor = functions.create_lable_for_loss_tensor(batch_label_index, end_start_index_label, device) # (b,L)
                loss_training_batch = criterion(batch_logits, lable_for_loss_tensor)  # (b,C,L), (b,L) --> tensor

            
            scaler.scale(loss_training_batch).backward()
            if (i + 1) % iter_gradient_accumulation == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()         
            loss_training_epoch += loss_training_batch.item()

            

        
        print('the training loss in epoch',epoch+1,'is:',loss_training_epoch/df_train_top500.shape[0])
        validation_loss, category_accuracy, first_class_accuracy, second_class_accuracy, disease_accuracy, valid_f1_micro, valid_f1_weighted, valid_AUROC_micro, valid_AUROC_weighted, valid_precision_micro, valid_recall_micro= valid(validation_dataloader)
        # wandb.log({'training_loss': loss_training_epoch/df_train_top500.shape[0]})
        # wandb.log({'validation_loss': validation_loss.item()})
        # wandb.log({'valid_category_accuracy': category_accuracy})
        # wandb.log({'valid_first_class_accuracy': first_class_accuracy})
        # wandb.log({'valid_second_class_accuracy': second_class_accuracy})
        # wandb.log({'valid_disease_accuracy': disease_accuracy})
        # wandb.log({'valid_f1_micro': valid_f1_micro.item()})
        # wandb.log({'valid_f1_weighted': valid_f1_weighted.item()})
        # wandb.log({'valid_AUROC_micro': valid_AUROC_micro.item()})
        # wandb.log({'valid_AUROC_weighted': valid_AUROC_weighted.item()})
        # wandb.log({'valid_precision_micro': valid_precision_micro.item()})
        # wandb.log({'valid_recall_micro': valid_recall_micro.item()})

        print('the valid loss in epoch',epoch+1,'is:', validation_loss.item())
        print('the valid accuracy in epoch',epoch+1,'is:', category_accuracy, first_class_accuracy, second_class_accuracy, disease_accuracy)
        print('the valid f1 micro in epoch',epoch+1,'is:', valid_f1_micro.item())
        print('the valid f1 weighted in epoch',epoch+1,'is:', valid_f1_weighted.item())
        print('the valid AUROC micro in epoch',epoch+1,'is:', valid_AUROC_micro.item())
        print('the valid AUROC weighted in epoch',epoch+1,'is:', valid_AUROC_weighted.item())
        print('the valid_precision micro in epoch',epoch+1,'is:', valid_precision_micro.item())
        print('the valid_recall_micro weighted in epoch',epoch+1,'is:', valid_recall_micro.item())
    
    print('Training Finished')



In [ ]:
# mp.set_start_method('spawn',force=True)
train(train_dataloader, validation_dataloader, epochs=args.epoch)

In [ ]:
torch.save(transformer_model, './trained_model/train02.pth')

In [ ]:
# click  argparse
# mix precision  √
# gradient accumulation  √
# gpt2 
# freeze bert  √
# generation in batch

In [ ]:
# generation : which maks was used?
# gradient accumulation and mix precision right?